In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Installing Necessary Libraries

In [ ]:
!pip install nltk
!pip install pandas
!pip install torch
!pip install chromadb
!pip install sentence-transformers
!pip install whisper
!pip install scikit-learn
!pip install openai-whisper

^C


### Importing Libraries

In [2]:
import sqlite3
import pandas as pd
import zipfile
import io
import re
import nltk
import torch
import random
import string
from nltk.corpus import stopwords
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import chromadb
from chromadb.utils import embedding_functions
from nltk.tokenize import word_tokenize

### Reading the Data from 'eng_subtitles_database.db'

In [3]:
# Read the code below and write your observation in the next cell
path ='/content/drive/MyDrive/Enhancing-Search-Engine-Relavance-for-Video-Subtitles/Data/eng_subtitles_database.db'
conn = sqlite3.connect(path)
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
print(cursor.fetchall())

[('zipfiles',)]


### Finding the names of the columns

In [4]:
cursor.execute("PRAGMA table_info('zipfiles')")
cols = cursor.fetchall()
for col in cols:
    print(col[1])

num
name
content


### Storing the data in pandas DataFrame

In [5]:
df = pd.read_sql_query("""SELECT * FROM zipfiles""", conn)
df.head()

,num,name,content
0,9180533,the.message.(1976).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x1c\xa9\x...
1,9180583,here.comes.the.grump.s01.e09.joltin.jack.in.bo...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x17\xb9\x...
2,9180592,yumis.cells.s02.e13.episode.2.13.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00L\xb9\x99V...
3,9180594,yumis.cells.s02.e14.episode.2.14.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00U\xa9\x99V...
4,9180600,broker.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x001\xa9\x99V...


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82498 entries, 0 to 82497
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   num      82498 non-null  int64 
 1   name     82498 non-null  object
 2   content  82498 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.9+ MB


In [7]:
print(df.iloc[0, 2])
# here 2 represent the index of content column
# 0 represents the row number

b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x1c\xa9\x99V\x9fx\x96\xf0\x8c\x9e\x00\x00\x86\x9b\x01\x00;\x00\x00\x00The.Message.1976.REMASTERED.1080p.BluRay.x264-PiGNUS.EN.srt\xad\xbdm\x93\xdc\xc6\x91.\xfa\x9d\x11\xfc\x0f-}\xe1=\x11-\x9d\x06P\x85\x17\x9d\x8d\xd5%%[\xa4-Y>&u\x15>\xdf\xd0\xd3\x98\x19x\xfae\x0cts<\xfe\xf57\x9f\'\xb3\n\xd9\xa4\xbc\xbb\xf7\xc6Fl\xacELW\xa2\xaa\x90\x95\x95\xafO\x16/_l6\xdf\xe0\xff\xea\xf5f\xb3Y}\xf5\xd5\xbf\xaf\xf4AQ\xae7Mx\xf9\xe2\xd7\xfe|s\xbf\xea\x8f\xcf\xab\x8f\xe3n8\xadN\xc7\xfdx\x1cVO\xe3\xf9~\xf5\xf3\xe3p\xfc\xea\xfd/o>\xbc\xfb\xf0\xe3\xef\xde\xbf|\xf1\xfbi\x18Vo\xa6\xd3\xd3<L\xab\xe1\x1f\xe7\xe18\x8f\xa7\xe37\xab\xd3\xbc\xdb~-\xc3\x1e\xfe\xa7<|\xf9\xe2\xe5\x8bR_[~S\xd6\xeb\xa2k\xf3k\xe5A\xb7\xeeb\xf5\xf2\xc5\xbb\xe3\xea|?\xac\x8e\xfdaX\x9dnW?\x9cvk>8\x9c\xe6\xf3\xean\xeao\xc6\xd3ev\x8f~\x1a\xa6\x9b\xf1\xf6\xb2\xff\x1a\xe4\xabD\xbe*d\x11\xa5#_U\xeb\xaa\xd9`\xa6\xa7\xc3\xea\xa7\xcb}\x7f8\xf4F\xf9\xa7a\x9e\x87\xe3\x9d\xcc\\\xdf\x07B!\x13\xaa\xd61n<!\xd9\xaf\xd0\

In [8]:
print(df['content'])

0        b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x1c\xa9\x...
1        b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x17\xb9\x...
2        b'PK\x03\x04\x14\x00\x00\x00\x08\x00L\xb9\x99V...
3        b'PK\x03\x04\x14\x00\x00\x00\x08\x00U\xa9\x99V...
4        b'PK\x03\x04\x14\x00\x00\x00\x08\x001\xa9\x99V...
                               ...                        
82493    b'PK\x03\x04\x14\x00\x00\x00\x08\x00\xb8\xa6\x...
82494    b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x13\x97\x...
82495    b'PK\x03\x04\x14\x00\x00\x00\x08\x00$\x97\x9aV...
82496    b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x00\x97\x...
82497    b'PK\x03\x04\x14\x00\x00\x00\x08\x00,\x97\x9aV...
Name: content, Length: 82498, dtype: object


In [9]:
df = df.sample(frac=0.25, random_state=42) #taking only 30% data randomly from df since the dataset is large

### Decoding the content

In [10]:
count = 0

def decode_method(binary_data):
    global count
    # Decompress the binary data using the zipfile module
    # print(count, end=" ")
    count += 1
    with io.BytesIO(binary_data) as f:
        with zipfile.ZipFile(f, 'r') as zip_file:
            # Assuming there's only one file in the ZIP archive
            subtitle_content = zip_file.read(zip_file.namelist()[0])

    # Now 'subtitle_content' should contain the extracted subtitle content
    return subtitle_content.decode('latin-1')  # Assuming the content is UTF-8 encoded text

In [11]:
df['content'] = df['content'].apply(decode_method)

df.head()

,num,name,content
17262,9251120,maybe.this.time.(2014).eng.1cd,"ï»¿1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch..."
7294,9211589,down.the.shore.s01.e10.and.justice.for.all.(19...,"1\r\n00:00:09,275 --> 00:00:11,876\r\n¶ Oh, I ..."
47707,9380845,uncontrollably.fond.s01.e07.heartache.(2016).e...,"1\r\n00:00:07,140 --> 00:00:14,220\r\n<i>Timin..."
29914,9301436,screen.two.s13.e04.the.precious.blood.(1996).e...,"1\r\n00:00:06,133 --> 00:00:08,900\r\n[etherea..."
54266,9408707,battlebots.(2015).eng.1cd,"ï»¿1\r\n00:00:01,480 --> 00:00:03,570\r\n[Chri..."


In [12]:
df['content']

,content
17262,"ï»¿1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch..."
7294,"1\r\n00:00:09,275 --> 00:00:11,876\r\n¶ Oh, I ..."
47707,"1\r\n00:00:07,140 --> 00:00:14,220\r\n<i>Timin..."
29914,"1\r\n00:00:06,133 --> 00:00:08,900\r\n[etherea..."
54266,"ï»¿1\r\n00:00:01,480 --> 00:00:03,570\r\n[Chri..."
...,...
17345,"ï»¿1\r\n00:00:01,002 --> 00:00:02,346\r\n<i>Pr..."
64569,"ï»¿1\r\n00:00:02,402 --> 00:00:03,680\r\nANNAL..."
1941,"ï»¿1\r\n00:00:04,304 --> 00:00:07,207\r\nWashi..."
74537,"1\r\n00:00:06,310 --> 00:00:09,890\r\n..24 str..."


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20624 entries, 17262 to 67720
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   num      20624 non-null  int64 
 1   name     20624 non-null  object
 2   content  20624 non-null  object
dtypes: int64(1), object(2)
memory usage: 644.5+ KB


In [14]:
df.size

61872

In [15]:
df.shape

(20624, 3)

### Data Cleaning (Removing time stamps and unwanted chars)

In [16]:
def clean_text(doc):
    # Remove timestamps (e.g., [00:01:30] and 00:01:30 --> 00:01:35)
    doc = re.sub(r"\[.*?\]", "", doc)  # Remove [timestamps]
    doc = re.sub(r"\d{2}:\d{2}:\d{2},\d{3} --> \d{2}:\d{2}:\d{2},\d{3}", "", doc)  # Remove timestamp ranges

    # Remove formatting artifacts (e.g., ¶, ï»¿, â™, etc.)
    doc = re.sub(r"[ï»¿¶â™ªâ€œâ€â€™]", "", doc)

    # Remove subtitle separators (-->), HTML tags, and newlines
    doc = re.sub(r"-->", " ", doc)  # Remove subtitle arrows
    doc = re.sub(r"<.*?>", "", doc)  # Remove <HTML tags>
    doc = re.sub(r"\r\n", " ", doc)  # Normalize newlines

    # Remove punctuation, special characters, and numbers
    doc = "".join([char for char in doc if char not in string.punctuation and not char.isdigit()])

    # Convert text to lowercase and remove extra spaces
    doc = doc.lower().strip()
    doc = re.sub(r"\s+", " ", doc)  # Normalize spaces

    return doc

df['content'] = df['content'].apply(clean_text)


In [17]:
print(df['content'])

17262    watch any video online with opensubtitles free...
7294     oh i know that its getting late but i dont wan...
47707    timing and subtitles by the uncontrollable lov...
29914    apiopensubtitlesorg is deprecated please imple...
54266    oh no not the minibots oh you leave those litt...
                               ...                        
17345    previously on alias marshall counted sources t...
64569    annalise previously on how to get away with mu...
1941     washington square park manhattan on the far si...
74537    strangers were marooned in remote samoa and he...
67720    you know what the offer was this is the quicke...
Name: content, Length: 20624, dtype: object


In [18]:
df

,num,name,content
17262,9251120,maybe.this.time.(2014).eng.1cd,watch any video online with opensubtitles free...
7294,9211589,down.the.shore.s01.e10.and.justice.for.all.(19...,oh i know that its getting late but i dont wan...
47707,9380845,uncontrollably.fond.s01.e07.heartache.(2016).e...,timing and subtitles by the uncontrollable lov...
29914,9301436,screen.two.s13.e04.the.precious.blood.(1996).e...,apiopensubtitlesorg is deprecated please imple...
54266,9408707,battlebots.(2015).eng.1cd,oh no not the minibots oh you leave those litt...
...,...,...,...
17345,9251353,alias.s05.e09.the.horizon.(2005).eng.1cd,previously on alias marshall counted sources t...
64569,9448295,how.to.get.away.with.murder.s02.e15.anna.mae.(...,annalise previously on how to get away with mu...
1941,9189670,the.sandman.s01.e08.the.sound.of.her.wings.(20...,washington square park manhattan on the far si...
74537,9488484,australian.survivor.s04.e02.episode.4.2.(2019)...,strangers were marooned in remote samoa and he...


In [ ]:
df.head(10)

In [19]:
df.to_csv('/content/drive/MyDrive/Enhancing-Search-Engine-Relavance-for-Video-Subtitles/Data/eng_subtitles.csv',index=False)

### Chunking function

In [ ]:
chunk_size  = 500
overlap =100

def chunk_text(text):
    chunks = []
    start =0
    while start < len(text):
        chunk = text[start:start + chunk_size]
        chunks.append(chunk.lower())
        start+=chunk_size -overlap
    return chunks

### Storing the embeddings in chromadb


In [ ]:
def store_embeddings_in_chromadb(subtitles_df):

    print("Initializing ChromaDB...")
    client = chromadb.PersistentClient(path="/content/drive/MyDrive/chroma_search_engine_sub_db")
    collection = client.get_or_create_collection(name="c_sub_embeddings")

    print("Loading embedding model...")
    model = SentenceTransformer("all-MiniLM-L6-v2", device='cuda' if torch.cuda.is_available() else 'cpu')

    print("Cleaning text...")
    subtitles_df['cleaned_content'] = subtitles_df['content']

    batch_size = 100
    print(f"Processing {len(subtitles_df)} subtitles in batches of {batch_size}...")

    for start in range(0, len(subtitles_df), batch_size):
        print(f"Processing batch {start} to {start + batch_size}...")
        end = start + batch_size
        batch = subtitles_df.iloc[start:end]

        all_chunks, all_ids, all_metadatas = [], [], []

        for num, name, content in zip(batch['num'], batch['name'], batch['cleaned_content']):
            chunks = chunk_text(content)
            all_chunks.extend(chunks)
            all_ids.extend([f"{num}_{i}" for i in range(len(chunks))])
            all_metadatas.extend([{"name": name, "content": chunk} for chunk in chunks])

        print(f"Encoding {len(all_chunks)} chunks...")
        with torch.no_grad():
            embeddings = model.encode(all_chunks, batch_size=32, show_progress_bar=True).tolist()

        print("Adding embeddings to ChromaDB...")
        collection.add(
            ids=all_ids,
            embeddings=embeddings,
            metadatas=all_metadatas
        )

    print("Embedding storage complete.")
    return collection

In [ ]:
collection = store_embeddings_in_chromadb(df)

Available RAM: 13.61 GB
